In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2024/sample_submission.json
/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json


In [2]:
import json

training_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json'
evaluation_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json'
evaluation_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json'
sample_submission_path = '/kaggle/input/arc-prize-2024/sample_submission.json'
training_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json'
test_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'

#function to load JSON data
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

#load each dataset
training_solutions = load_json_data(training_solutions_path)
evaluation_solutions = load_json_data(evaluation_solutions_path)
evaluation_challenges = load_json_data(evaluation_challenges_path)
sample_submission = load_json_data(sample_submission_path)
training_challenges = load_json_data(training_challenges_path)
test_challenges = load_json_data(test_challenges_path)

In [3]:
#inspect the structure of the data
def inspect_data(data, name):
    print(f"Inspecting {name}:")
    if isinstance(data, list):
        print(f"Number of items: {len(data)}")
        if len(data) > 0:
            print(f"Example item: {data[0]}")
    elif isinstance(data, dict):
        print(f"Number of keys: {len(data.keys())}")
        if len(data.keys()) > 0:
            first_key = list(data.keys())[0]
            print(f"Example item under key '{first_key}': {data[first_key]}")
    else:
        print("Unknown data type")
    print("\n")

inspect_data(training_solutions, "Training Solutions")
inspect_data(evaluation_solutions, "Evaluation Solutions")
inspect_data(evaluation_challenges, "Evaluation Challenges")
inspect_data(sample_submission, "Sample Submission")
inspect_data(training_challenges, "Training Challenges")
inspect_data(test_challenges, "Test Challenges")

Inspecting Training Solutions:
Number of keys: 400
Example item under key '007bbfb7': [[[7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 7, 0, 7, 7, 0, 0, 0, 0]]]


Inspecting Evaluation Solutions:
Number of keys: 400
Example item under key '00576224': [[[3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8], [2, 3, 2, 3, 2, 3], [8, 7, 8, 7, 8, 7], [3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8]]]


Inspecting Evaluation Challenges:
Number of keys: 400
Example item under key '00576224': {'test': [{'input': [[3, 2], [7, 8]]}], 'train': [{'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}, {'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 

In [4]:
def get_max_grid_size(challenges, solutions):
    max_input_height, max_input_width = 0, 0
    max_output_height, max_output_width = 0, 0
    
    for key in challenges.keys():
        challenge = challenges[key]
        for example in challenge['train']:
            input_grid = example['input']
            output_grid = example['output']
            max_input_height = max(max_input_height, len(input_grid))
            max_input_width = max(max_input_width, len(input_grid[0]))
            max_output_height = max(max_output_height, len(output_grid))
            max_output_width = max(max_output_width, len(output_grid[0]))
        for test_case in challenge['test']:
            test_input = test_case['input']
            max_input_height = max(max_input_height, len(test_input))
            max_input_width = max(max_input_width, len(test_input[0]))
            #assuming test_output size can be derived similarly

    return max_input_height, max_input_width, max_output_height, max_output_width

max_input_height, max_input_width, max_output_height, max_output_width = get_max_grid_size(training_challenges, training_solutions)
print(f"Max input size: {max_input_height}x{max_input_width}, Max output size: {max_output_height}x{max_output_width}")

Max input size: 30x30, Max output size: 30x30


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ARCDataset(Dataset):
    def __init__(self, challenges, solutions, max_size, transform=None):
        self.data = []
        self.max_size = max_size
        self.transform = transform
        for key in challenges.keys():
            challenge = challenges[key]
            solution = solutions[key]
            for example in challenge['train']:
                input_grid = example['input']
                output_grid = example['output']
                self.data.append((input_grid, output_grid))
            for test_case in challenge['test']:
                test_input = test_case['input']
                #use the corresponding solution as the target output
                test_output = solution[len(self.data) % len(solution)]
                self.data.append((test_input, test_output))

    def __len__(self):
        return len(self.data)

    def pad_grid(self, grid):
        padded_grid = np.zeros(self.max_size)
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                padded_grid[i][j] = grid[i][j]
        return padded_grid

    def __getitem__(self, idx):
        input_grid, output_grid = self.data[idx]
        input_grid = self.pad_grid(input_grid)
        output_grid = self.pad_grid(output_grid)
        input_grid = torch.tensor(input_grid, dtype=torch.float32).unsqueeze(0)
        output_grid = torch.tensor(output_grid, dtype=torch.float32).unsqueeze(0)
        if self.transform:
            input_grid = self.transform(input_grid)
            output_grid = self.transform(output_grid)
        return input_grid, output_grid

#define the maximum size
max_size = (30, 30)

#create datasets with padding
transform = transforms.Compose([transforms.Lambda(lambda x: x)])  #no additional transform needed
training_dataset = ARCDataset(training_challenges, training_solutions, max_size, transform=transform)
evaluation_dataset = ARCDataset(evaluation_challenges, evaluation_solutions, max_size, transform=transform)

#create dataloaders
training_loader = DataLoader(training_dataset, batch_size=1, shuffle=True)
evaluation_loader = DataLoader(evaluation_dataset, batch_size=1, shuffle=False)

In [6]:
#perception module
import torch.nn.functional as F
import torch.nn as nn

class PerceptionModule(nn.Module):
    def __init__(self):
        super(PerceptionModule, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        return x

#instantiate and test the module
perception_model = PerceptionModule()
inputs = torch.randn(1, 1, 8, 8)
outputs = perception_model(inputs)

In [7]:
#memory module
class MemoryModule(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MemoryModule, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        #LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        #fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        #reshape x to (batch_size, seq_len, features)
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, height * width, channels)  #(batch_size, seq_len, features)
        
        #LSTM input: (batch_size, seq_len, input_size)
        x, _ = self.lstm(x)
        
        #take the last time step's output
        x = self.fc(x[:, -1, :])  #(batch_size, output_size)
        
        return x

memory_module = MemoryModule(input_size=32, hidden_size=64, num_layers=1, output_size=30*30)
inputs = torch.randn(1, 32, 4, 4)
outputs = memory_module(inputs)

In [8]:
#creativity module
class CreativityModule(nn.Module):
    def __init__(self, input_size):
        super(CreativityModule, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
creativity_model = CreativityModule(input_size=128)
inputs = torch.randn(1, 128) 
outputs = creativity_model(inputs)

In [9]:
#reasoning module
class ReasoningModule(nn.Module):
    def __init__(self):
        super(ReasoningModule, self).__init__()
        self.fc1 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 30 * 30)  #output grid size is 30x30

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = x.view(-1, 30, 30)  #reshape to match output grid size
        return x

In [10]:
#compassionate module
class CompassionateModule(nn.Module):
    def __init__(self):
        super(CompassionateModule, self).__init__()
        self.fc1 = nn.Linear(30*30, 64)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
#combine the modules above into the Yambi Model
class Yambi(nn.Module):
    """
    Yambi is an AI model designed to solve abstract reasoning tasks by integrating various cognitive abilities. 
    It consists of five main modules:
    
    1. Perception Module: Processes and understands input grids using convolutional neural networks.
    2. Memory Module: Stores and recalls information about previously seen tasks using LSTM networks.
    3. Creativity Module: Generates creative solutions and explores novel approaches using a latent space.
    4. Reasoning Module: Applies logical reasoning to infer solutions based on perceived information.
    5. Compassionate Module: Analyzes emotional context and adjusts responses accordingly.

    These modules work together to perceive, remember, create, reason, and empathize, aiming to solve abstract reasoning tasks.
    """

    def __init__(self, perception_model, memory_model, creativity_model, reasoning_model, compassionate_model):
        super(Yambi, self).__init__()
        self.perception_model = perception_model
        self.memory_model = memory_model
        self.creativity_model = creativity_model
        self.reasoning_model = reasoning_model
        self.compassionate_model = compassionate_model
    
    def forward(self, x):
        #Perception Module
        perception_output = self.perception_model(x)

        #Memory Module
        memory_output = self.memory_model(perception_output)

        #Creativity Module
        creativity_output = self.creativity_model(memory_output)

        #Reasoning Module
        reasoning_output = self.reasoning_model(creativity_output)

        #Compassionate Module
        compassionate_output = self.compassionate_model(reasoning_output)

        return reasoning_output, compassionate_output

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
#instantiate Perception Module
perception_model = PerceptionModule()

#instantiate Memory Module with appropriate input_size based on Perception Module output
perception_output_size = 32
memory_model = MemoryModule(input_size=perception_output_size, hidden_size=128, num_layers=1, output_size=10)

#instantiate Creativity Module with appropriate input_size based on Memory Module output
creativity_input_size = 10 
creativity_model = CreativityModule(input_size=creativity_input_size)

#instantiate Reasoning Module
reasoning_model = ReasoningModule()

#instantiate Compassionate Module
compassionate_model = CompassionateModule()

#instantiate the Yambi model
yambi_model = Yambi(perception_model, memory_model, creativity_model, reasoning_model, compassionate_model).to(device)

#test the Yambi model with dummy input to check the flow and shape
dummy_input = torch.randn(1, 1, 6, 7).to(device)
output_reasoning, output_compassionate = yambi_model(dummy_input)

In [14]:
import torch.optim as optim

In [15]:
#number of epochs for training
num_epochs = 30

#define the criterion and optimizer
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(yambi_model.parameters(), lr=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  #reduce LR by 0.1 every 10 epochs

#training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    
    #iterate over the data in the training loader
    for inputs, labels in training_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        #zero the parameter gradients
        optimizer.zero_grad()
        
        #forward pass
        reasoning_output, compassionate_output = yambi_model(inputs)
        
        #squeeze the labels to remove the extra dimension if needed
        labels = labels.squeeze(1)
        
        #compute the CrossEntropyLoss
        loss = criterion(reasoning_output, labels)
        
        #backward pass and optimize
        loss.backward()
        optimizer.step()
        
        #accumulate the loss
        running_loss += loss.item()

    #print the average loss for the epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(training_loader)}")

print('Training complete.')

Epoch [1/30], Loss: 23.22827790195298
Epoch [2/30], Loss: 21.127534937126534
Epoch [3/30], Loss: 20.592264998699857
Epoch [4/30], Loss: 20.43893636666169
Epoch [5/30], Loss: 20.36795857850987
Epoch [6/30], Loss: 20.325896390339704
Epoch [7/30], Loss: 20.29301555284079
Epoch [8/30], Loss: 20.260717333267348
Epoch [9/30], Loss: 20.233371241733742
Epoch [10/30], Loss: 20.203766885542066
Epoch [11/30], Loss: 20.16666042998572
Epoch [12/30], Loss: 20.143667442268747
Epoch [13/30], Loss: 20.121581258130835
Epoch [14/30], Loss: 20.107264555708806
Epoch [15/30], Loss: 20.097204805907985
Epoch [16/30], Loss: 20.083505067939633
Epoch [17/30], Loss: 20.074498046822715
Epoch [18/30], Loss: 20.06775835139744
Epoch [19/30], Loss: 20.054122216912795
Epoch [20/30], Loss: 20.035099843521284
Epoch [21/30], Loss: 20.039191389170163
Epoch [22/30], Loss: 20.02384718035687
Epoch [23/30], Loss: 20.02839695344808
Epoch [24/30], Loss: 20.010518088054273
Epoch [25/30], Loss: 20.006338095554824
Epoch [26/30], Lo

In [16]:
#evaluate the Yambi model and calculate the ARC score
def calculate_arc_score(model, evaluation_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in evaluation_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).long()

            reasoning_output, compassionate_output = model(inputs)
            predicted_output = torch.argmax(reasoning_output, dim=1)
            labels = labels.squeeze(1)
            
            correct += (predicted_output == labels).sum().item()
            total += labels.numel()

    return correct / total

#evaluate the model on the evaluation dataset
arc_score = calculate_arc_score(yambi_model, evaluation_loader, device)
print(f'ARC Score: {arc_score * 100:.2f}%')

ARC Score: 4.91%
